In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as pdr
import datetime 
import json

In [2]:
# Input
start_date = datetime.datetime(2016,1,1)
end_date = datetime.datetime.today()

# 10 Stocks of same sector are chosen as representative of that sector
# tickers = ['BAJAJ-AUTO.NS','TCS.NS','INFY.NS',]
nstocks = 10
sector = 'Utilities'
sector_data = pd.read_csv(f'./sector_data/{sector}.csv',index_col=0)
tickers = list(sector_data.nlargest(nstocks,'MktCap')['Ticker'])

data = pdr.get_data_yahoo(tickers,start_date,end_date)

In [3]:
data = data.dropna()
data.tail()

Attributes   Adj Close                                                    \
Symbols        NTPC.NS POWERGRID.NS TATAPOWER.NS JSWENERGY.NS    NHPC.NS   
Date                                                                       
2022-09-07  167.300003   224.399994   246.949997   352.799988  38.049999   
2022-09-08  166.800003   224.399994   244.899994   351.600006  37.650002   
2022-09-09  166.600006   222.899994   242.350006   348.950012  36.700001   
2022-09-12  166.399994   223.949997   242.800003   348.700012  36.950001   
2022-09-13  167.399994   226.800003   241.050003   344.049988  37.000000   

Attributes                                                          ...  \
Symbols    TORNTPOWER.NS SJVN.NS NLCINDIA.NS    CESC.NS  RPOWER.NS  ...   
Date                                                                ...   
2022-09-07    580.799988   30.85   81.650002  84.199997  21.650000  ...   
2022-09-08    573.450012   31.25   81.000000  82.900002  21.299999  ...   
2022-09-09    565.700012   31.35   78.750000  83.250000  19.200001  ...   
2022-09-12    560.950012   31.60   78.150002  85.449997  21.100000  ...   
2022-09-13    552.650024   31.75   77.650002  84.349998  20.049999  ...   

Attributes      Volume                                                     \
Symbols        NTPC.NS POWERGRID.NS TATAPOWER.NS JSWENERGY.NS     NHPC.NS   
Date                                                                        
2022-09-07  19415892.0    5900059.0   30474615.0    1618180.0  10521195.0   
2022-09-08  11595205.0    5390802.0   14951235.0    1130766.0  10207654.0   
2022-09-09  16415150.0    6871521.0   12290186.0     875563.0  14983024.0   
2022-09-12  15469466.0    6584190.0   10322525.0     785552.0  11267951.0   
2022-09-13  15641679.0    8458481.0   13852794.0     932274.0  18015095.0   

Attributes                                                               
Symbols    TORNTPOWER.NS    SJVN.NS NLCINDIA.NS    CESC.NS    RPOWER.NS  
Date                                                                     
2022-09-07      858247.0  1974656.0   5842934.0  7705238.0   64869145.0  
2022-09-08      300802.0  2848422.0  10071812.0  2584014.0   48143250.0  
2022-09-09      328987.0  2673363.0   4197554.0  2084297.0  115341202.0  
2022-09-12      565980.0  2247045.0   1974283.0  5181737.0  107892593.0  
2022-09-13      347210.0  3038978.0   3337920.0  1920324.0   20768831.0  

[5 rows x 60 columns]

In [4]:
data.index

DatetimeIndex(['2016-01-01', '2016-01-04', '2016-01-05', '2016-01-06',
               '2016-01-07', '2016-01-08', '2016-01-11', '2016-01-12',
               '2016-01-13', '2016-01-14',
               ...
               '2022-08-30', '2022-09-01', '2022-09-02', '2022-09-05',
               '2022-09-06', '2022-09-07', '2022-09-08', '2022-09-09',
               '2022-09-12', '2022-09-13'],
              dtype='datetime64[ns]', name='Date', length=1655, freq=None)

In [5]:
temp_list = []
for i in tickers:
    temp_list.append(np.array([data['Adj Close'][i]]).T)

In [6]:
X = np.block([temp_list]).T

n = number of companies in a given portfolio  
m = number of data snapshots taken  
#### size of X is (n x m)  
Trading algorithm developed is parametrized into 2 key parameters  

**mp (1,24) = number of past days of market snapshot data taken**  
**mf (1,10) = number of days in the future predicted**

Use specific portions of the data matrix X as specified by mp and mf
Look for trading hotspots or regions of (mp,mf) where predictions are optimal

### If there is a sufficient predicted change in the stock (at least 1%), then trade is executed  
1. Initial investment is 10,00,000.
2. Transaction cost is 20 for each position.
3. All money is invested evenly across all companies in portfolio

As it is self financing strategy, the purchase of a new asset must be financed by the sale of an old one.

### Hotspot
1. Prediction accuracy > 53% at that location.
2. Avg predition score of all 9 surrounding cells > 53%.

Now we focus on 2 main things
1. Training step in the algo to decide (mp,mf).
2. Implementation of the trading based on the results

prediction accuracy = (TrueNegativities+TruePositive)/(TrueNegative+TruePositive+FalseNegative+FalsePositive)

Eq 1.16 and 1.24 from Kutz book

So basically we want to predict states based on historical value.
$$ X_2 = AX_1 $$
where $X_2$ is the time shifted matrix of $X_1$ i.e. $X_1=X[:-1]$ and $X_2=X[1:]$.  

### DMD
Outputs and inputs are 
$$ [\Phi ,\Omega ,\lambda ,b,X_{DMD}] = DMD(X_1,X_2,r,dt) $$
Start by taking economic SVD of X1 and compute its main components
$$ [U,s,V_h]=np.linalg.svd(X_1,full\_matrices=False) $$  
$$ r = np.min(r, np.shape(U)[1]) $$
$$ U_r,S_r,V_r=U[:,:r],np.diag(s[:r]),V_h[:r,:]' $$

Using these main components compute $\tilde{A}$, which is a low dimensional linear model, since computing $A$ by taking pseudo inverse of $X_1$ is computationally expensive.
$$ \tilde{A} = U_r'X_2V_r/S_r $$

Compute eig of $\tilde{A}$ and use it to compute DMD modes i.e. eigenvectors of $A$, $\Phi$
$$ [d, W_r] = np.linalg.eig(\tilde{A}) $$
$$ \Phi = X_2V_r/S_rW_r $$

Hence 
$$ \lambda = np.diag(d) $$
$$ \Omega = log(\lambda)/dt $$
$$ x(t) \approx \Phi e^{\Omega t}b $$
where $b$ is the vector of initial amplitude of each mode. We can find $b$ as at time $t=0$
$$ X1[:,0] = \Phi b $$
$$ b = X1[:,0] np.linalg.pinv(\Phi) $$
$$ t = timearray(dtSpaced(butNotNecessary)) = size(something,1,1)$$
$$ X_{DMD} = \Phi e^{\Omega t}b =(ReshapingRequired)$$


In [7]:
def DMD(X,r,dt,mf):
    '''
    Inputs:
        X = numpy.ndarray: Historical time series data with time along the columns and current time being the last column
        r = scalar: Number of main modes to consider
        dt = scaler: time step between each column
        mf = scalar: Number of future time steps you want to predict
    Outputs:
        X_DMD = mf+1 columns of predicted time series data with first column being the current time step
    '''
    X1,X2 = X[:,:-1],X[:,1:] # Last column of X is current price of the day close to closing
    U,s,Vh = np.linalg.svd(X1,full_matrices=False)
    r = min(r,np.shape(U)[1])
    Ur,Sr,Vr = U[:,:r],np.diag(s[:r]),Vh[:r,:].T
    A_T = Ur.T@X2@Vr@np.linalg.inv(Sr)
    d,Wr = np.linalg.eig(A_T)
    Phi = X2@Vr@np.linalg.inv(Sr)@Wr
    Lambda = np.diag(d)
    Omega = np.log(Lambda)/dt
    b = np.linalg.pinv(Phi)@X[:,-1:]
    
    t = np.arange(0,(mf+1)*dt,dt)
    t = t.reshape((t.shape[0],1,1))
    time_dynamics = np.exp(t*Omega)@b
    X_DMD = (Phi@time_dynamics).T.reshape((Phi.shape[0],t.shape[0]))
    return X_DMD
    

In [8]:
## Assign train data
X_train = X[:,:int(X.shape[1]/3)]
X_test = X[:,int(X.shape[1]/3):]
X_train.shape

(10, 551)

In [9]:
def backtest(mp,mf,X_inp):
    ## Initialize variables
    #### When did we buy and at what price, when did we sell and at what price
    bank_balance = 1000000
    #### Queues to store at what index we bought that stock
    q = [np.array([]) for i in range(len(tickers))]
    #### trade summary for every stock 
    trade_summary = [pd.DataFrame(columns=['BuyDate_index','BuyPrice','SellDate_index','SellPrice','Lots']) for i in range(len(tickers))] # For 1 stock, make similar structure for other stocks as well
    #### Bank balance is distributed equally among stocks
    balance_per_stock = np.ones((len(tickers),1))*bank_balance/len(tickers)

    ## Begin for loop from max(mp) to train-max(mf)
    for i in range(24,X_inp.shape[1]-10):
        #### If bought at i-mf (check queue and remove if present) then sell
        for j in range(len(tickers)):
            if q[j].shape[0]!=0: # if queue is not empty
                if q[j][-1]==i-mf: # if last element is mf days before current then sell
                    ###### Sell means to pop queue, append P&L along with the iteration number and increase the bank balance again taking into account the P&L and transaction/brokerage cost (8 per trade)
                    q[j] = q[j][:-1]
                    ######## Update trade summary
                    trade_summary[j].iloc[len(trade_summary[j].index)-1][2:4] = [i,X_inp[j,i]]
                    ######## Update balance per stock
                    balance_per_stock[j] += X_inp[j,i]*trade_summary[j].iloc[len(trade_summary[j].index)-1][-1] # Include transaction costs
                
        #### If balance is there and DMD(...)[:,-1:] i.e. predicted price is greater than 1% then buy else do nothing
        dmd = DMD(X_inp[:,i-mp:i+1],5,1,mf)[:,-1:]
        for j in range(len(tickers)):
            if balance_per_stock[j]>=X_inp[j,i] and (dmd[j,0]-X_inp[j,i])/X_inp[j,i]>0.01:
                ## Buy 
                lots = int(balance_per_stock[j]/X_inp[j,i])
                #### Buy means append queue of possesions
                q[j] = np.append(q[j],i)
                #### Update trade summary
                trade_summary[j].loc[len(trade_summary[j].index)] = [i,X_inp[j,i],np.nan,np.nan,lots]
                #### Update (subtract) balance per stock
                balance_per_stock[j] -= lots*X_inp[j,i]

    return trade_summary
        ###### Buy means Bank balance is reduced queue is appended with the iteration number

In [10]:
range_mp = 24
range_mf = 10
trade_summary = np.zeros((range_mf,range_mp)).tolist()
for mp in range(1,range_mp+1):
    for mf in range(1,range_mf+1):
        trade_summary[mf-1][mp-1] = backtest(mp,mf,X_train)
        # print(trade_summary[mf-1][mp-1])
    print(mp)

1


d:\Programming\Python\Time_data_prediction\fin-env\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log
d:\Programming\Python\Time_data_prediction\fin-env\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in multiply
d:\Programming\Python\Time_data_prediction\fin-env\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in log
d:\Programming\Python\Time_data_prediction\fin-env\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [11]:
with open(f'./pre_processing_mp_mf/{sector}.json', 'w') as f:
    f.write(json.dumps([[[trade_summary[i][j][k].to_dict() for k in range(len(trade_summary[0][0]))] for j in range(len(trade_summary[0]))] for i in range(len(trade_summary))]))

In [12]:
## Do the above thing for all (kp,kf) pair and find %age of successful trades and plot colorplot
## This is done in post_peocessing.ipynb file

We define a trading hotspot as a particular pairing of $(m_p,m_f)$ integers such that  
1. $S(m_p,m_f)>0.53$, percentage of successful trades for a particular pair should be greater than 0.53.
2. $1/9\sum_{j=-1}^1\sum_{k=-1}^1 S_{m_p+k,m_f+j}>0.53$, basically average of the 9 surrounding cells should be greater than 0.53.

In [13]:
## Identify the trading hotspot
## Identified in post_processing.ipynb file

After trading hotspot is identified (if it is present), we can test the algorithm on the remaining data using the values of the trading hotspot.  

Else this particular sector is not feasible to trade in. Need to look for another sector for this strategy to work.

In [14]:
test_trade_summary = backtest(18,8,X_test)

d:\Programming\Python\Time_data_prediction\fin-env\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log
d:\Programming\Python\Time_data_prediction\fin-env\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide
d:\Programming\Python\Time_data_prediction\fin-env\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in multiply
d:\Programming\Python\Time_data_prediction\fin-env\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in log


In [15]:
profit_test = 0
success_test = 0
tot_trades_test = 0 # for a given (mp,mf) pair
for k in range(len(tickers)):
    curr_profit_summary = (test_trade_summary[k]['SellPrice']-test_trade_summary[k]['BuyPrice'])*test_trade_summary[k]['Lots']
    
    if len(curr_profit_summary)>=1 : 
        if curr_profit_summary[-1:].isnull().values.any(): 
            curr_profit_summary = curr_profit_summary[:-1]
            test_trade_summary[k] = test_trade_summary[k][:-1]
            # print('NaN encountered')
        success_test += len(curr_profit_summary[curr_profit_summary>test_trade_summary[k]['BuyPrice']*test_trade_summary[k]['Lots']/100]) #*100/len(curr_profit_summary)            
        tot_trades_test += len(curr_profit_summary)
    profit_test += np.sum(curr_profit_summary)
print('Profit', profit_test)
print('Success Trades',success_test,'; Total trades',tot_trades_test)
print('Success Percentage',success_test/tot_trades_test*100)

Profit 286683.66944885254
Success Trades 54 ; Total trades 111
Success Percentage 48.64864864864865
